In [12]:
!pip install fake_useragent

In [13]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from fake_useragent import UserAgent

ua = UserAgent()
headers = {'User-Agent': ua.chrome}

contents = requests.get("http://cheboksary.ginfo.ru/ulicy/", headers=headers).text
soup = BeautifulSoup(contents, 'lxml')

/usr/local/lib/python3.7/dist-packages/requests/__init__.py:91: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (3.0.4) doesn't match a supported version!
  RequestsDependencyWarning)


In [21]:
data_links = []
for i in soup.find_all("div", {"class": "street_unit"}):
    data_links.append([i.a.text, "http://cheboksary.ginfo.ru" + i.a["href"]])

In [23]:
contents = requests.get(data_links[1][1], headers=headers).text
soup = BeautifulSoup(contents, 'lxml')

In [41]:
from bs4 import BeautifulSoup
from joblib import Parallel, delayed
 
 
def parse(street, link):
    headers = {'User-Agent': ua.chrome}
    contents = requests.get(link, headers=headers).text
    soup = BeautifulSoup(contents, 'lxml')
    try:
        return street, [i.text for i in soup.find("div", {"class": "dom_list"})\
                        .find_all("a")]
    except:
        print(street, link)

result = Parallel(n_jobs=10,
                  verbose=True,
                  backend="multiprocessing")([delayed(parse)(street, link) for street, link in data_links])

[Parallel(n_jobs=10)]: Using backend MultiprocessingBackend with 10 concurrent workers.


40 лет Октября 4-я, ул. http://cheboksary.ginfo.ru/ulicy/4-ya_ulica_40_let_oktyabrya/
Авторемонтный, пр-д http://cheboksary.ginfo.ru/ulicy/avtoremontnyy_proezd/
Алатырское, ш. http://cheboksary.ginfo.ru/ulicy/alatyrskoe_shosse/
Аникеевская, ул. http://cheboksary.ginfo.ru/ulicy/anikeevskaya_ulica/


[Parallel(n_jobs=10)]: Done  30 tasks      | elapsed:    1.1s


Барбарисовая, ул. http://cheboksary.ginfo.ru/ulicy/barbarisovaya_ulica/
Вильямса, ул. http://cheboksary.ginfo.ru/ulicy/ulica_vilyamsa/
Восточный, пер. http://cheboksary.ginfo.ru/ulicy/vostochnyy_pereulok/
Еловая, ул. http://cheboksary.ginfo.ru/ulicy/elovaya_ulica/
Ермака 3-я, ул. http://cheboksary.ginfo.ru/ulicy/3-ya_ulica_ermaka/
Ермака 2-я, ул. http://cheboksary.ginfo.ru/ulicy/2-ya_ulica_ermaka/
Ермака 4-я, ул. http://cheboksary.ginfo.ru/ulicy/4-ya_ulica_ermaka/
Женьшеневая, ул. http://cheboksary.ginfo.ru/ulicy/zhenshenevaya_ulica/
Заводская, ул. http://cheboksary.ginfo.ru/ulicy/zavodskaya_ulica/
Заречная 2-я, ул. http://cheboksary.ginfo.ru/ulicy/2-ya_zarechnaya_ulica/
Инструментальный, пр-д http://cheboksary.ginfo.ru/ulicy/instrumentalnyy_proezd/
Историческая, наб. http://cheboksary.ginfo.ru/ulicy/istoricheskaya_naberezhnaya/
Ключевая, ул. http://cheboksary.ginfo.ru/ulicy/klyuchevaya_ulica/


[Parallel(n_jobs=10)]: Done 180 tasks      | elapsed:    5.7s


Марата 2-я, ул. http://cheboksary.ginfo.ru/ulicy/2-ya_ulica_marata/
Мичурина, ул. http://cheboksary.ginfo.ru/ulicy/ulica_michurina/
Молодогвардейская, ул. http://cheboksary.ginfo.ru/ulicy/molodogvardeyskaya_ulica/
МОПРа, ул. http://cheboksary.ginfo.ru/ulicy/ulica_mopra/
Мясокомбинатского проезда 1-я, линия http://cheboksary.ginfo.ru/ulicy/1-ya_liniya_myasokombinatskogo_proezda/
Мясокомбинатского проезда 4-я, линия http://cheboksary.ginfo.ru/ulicy/4-ya_liniya_myasokombinatskogo_proezda/
Мясокомбинатского проезда 2-я, линия http://cheboksary.ginfo.ru/ulicy/2-ya_liniya_myasokombinatskogo_proezda/
Мясокомбинатского проезда 6-я, линия http://cheboksary.ginfo.ru/ulicy/6-ya_liniya_myasokombinatskogo_proezda/
Мясокомбинатского проезда 5-я, линия http://cheboksary.ginfo.ru/ulicy/5-ya_liniya_myasokombinatskogo_proezda/
Нагорно-Рябиновская, ул. http://cheboksary.ginfo.ru/ulicy/nagorno-ryabinovskaya_ulica/
Николая Никольского, пр-т http://cheboksary.ginfo.ru/ulicy/prospekt_nikolaya_nikolskogo/
Озё

[Parallel(n_jobs=10)]: Done 430 tasks      | elapsed:   13.2s


Чапаева 2-я, ул. http://cheboksary.ginfo.ru/ulicy/2-ya_ulica_chapaeva/
Черничная, ул. http://cheboksary.ginfo.ru/ulicy/chernichnaya_ulica/
Черёмушки, ул. http://cheboksary.ginfo.ru/ulicy/ulica_cheremushki/


[Parallel(n_jobs=10)]: Done 504 out of 504 | elapsed:   15.4s finished


In [43]:
out = []
for i in result:
    if i:
        for j in i[1]:
            out.append([i[0], j])
out = pd.DataFrame(out, columns=["street", "house"])

In [45]:
out.to_csv("cheb_streets_houses.csv", index=False)